In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob

# Camera Calibration and Undistorting Images
In this notebook we are going to calibrate a monocular camera (i.e. estimate the camera matrix and distortion parameters) and use the calibration to undistort an image. We will start by using the standard OpenCV functions first and later implement the calibration and undistortion form scratch using numpy and scipy.
## Camera Calibration using OpenCV
I have printed out a [chessboard pattern](https://github.com/opencv/opencv/blob/master/doc/pattern.png) and took pictures of it using my phone's camera. For efficiency reasons I scaled down the images by a factor of 2. We will follow the [official OpenCV tutorial on camera calibration](https://docs.opencv.org/master/dc/dbb/tutorial_py_calibration.html).
<img src="data/IMG_1239.jpg" alt="drawing" width="30%"/>

In [ ]:
# Extract circle centers for all images. This might take a while...
image_files = glob.glob('data/*.jpg')

grid_size = 25 # mm
grid_shape = (9,6)

image_corners = np.zeros((0, grid_shape[0]*grid_shape[1] ,2))
chessboard_positions = np.zeros((grid_shape[0]*grid_shape[1],3))
chessboard_positions[:,:2] = np.mgrid[0:grid_shape[0],0:grid_shape[1]].T.reshape(-1,2)
chessboard_positions *= grid_size
image_shape = np.array([])

for file in image_files:
    img = cv2.cvtColor(cv2.imread(file), cv2.COLOR_BGR2GRAY)
    image_shape = img.shape
    ret, corners = cv2.findChessboardCorners(img, grid_shape)
    if ret:
        image_corners = np.append(image_corners, np.array(corners).reshape(1, -1, 2), axis=0)
    else:
        print(f"No chessboard detected in image {file}")
print(f"Processed {len(image_files)} images and found {image_corners.shape[0]} chessboards.")

In [ ]:
def calibrate_camera_openCV():
    object_points = np.tile(chessboard_positions, (image_corners.shape[0], 1, 1))
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points.astype(np.float32), image_corners.astype(np.float32), image_shape[::-1], None, None)
    return mtx, dist[0]

camera_mat_cv, dist_cv = calibrate_camera_openCV()
print("Camera matrix returned by OpenCV:")
with np.printoptions(suppress=True, precision=2):
    print(camera_mat_cv)
print("Distortion coefficients returned by OpenCV:")
print(f"k1: {dist_cv[0]:+.3e}; \tk2: {dist_cv[1]:+.3e}; \tp1: {dist_cv[2]:+.3e}; \tp2: {dist_cv[3]:+.3e}")